In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
spark = SparkSession.builder.appName('hello').getOrCreate()

In [2]:
path=f"gs://{'ornate-producer-329407'}/pyspark_nlp/CONSUMER_COMPLAINTS_TRAIN.csv"
path_class=f"gs://{'ornate-producer-329407'}/pyspark_nlp/labels.csv"

In [3]:
df1=spark.read.csv(path, header=True)
df2=spark.read.csv(path_class, header=True)

In [4]:
from pyspark.sql.functions import monotonically_increasing_id

df_1 = df1.withColumn("row_id", monotonically_increasing_id())
df_2 = df2.withColumn("row_id", monotonically_increasing_id())
df = df_1.join(df_2, ("row_id")).drop("row_id")

In [5]:
df.show()

+--------------------+--------------------+
|                text|               class|
+--------------------+--------------------+
|Duplicate payment...|Bank account or s...|
|I bought a money ...|Other financial s...|
|"I filed a proper...|    Credit reporting|
|My dispute result...|            Mortgage|
|As there is no su...|         Credit card|
|TransUnion failed...|Bank account or s...|
|TransUnion respon...|    Credit reporting|
|I have checked wi...|        Student loan|
|It is clear that ...|            Mortgage|
|         Please note|         Credit card|
|I have also inclu...|    Credit reporting|
|This list shows a...|    Credit reporting|
|My ex-wife and I ...|        Student loan|
|I 've had a credi...|     Debt collection|
|This is my fifth ...|            Mortgage|
|In an attempt to ...|Checking or savin...|
|My student loan i...|     Debt collection|
|On XX/XX/XXXX, I ...|         Payday loan|
|       On XX/XX/XXXX|    Credit reporting|
|I have not receiv...|Bank accou

In [6]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer


In [7]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stop_word_remover=StopWordsRemover(inputCol="token_text", outputCol="stop_tokens")
count_vec=CountVectorizer(inputCol="stop_tokens",outputCol="c_vec")
idf=IDF(inputCol="c_vec",outputCol="tf-idf")
class_to_num=StringIndexer(inputCol="class",outputCol="label")

In [8]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [9]:
cleaned= VectorAssembler(inputCols=['tf-idf'], outputCol='features')

In [10]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(labelCol='label',featuresCol='features')

In [11]:
from pyspark.ml import Pipeline

In [12]:
pipeline=Pipeline(stages=[
    class_to_num,
    tokenizer,
    stop_word_remover,
    count_vec,
    idf,
    cleaned,
    nb
])

In [13]:
pred=pipeline.fit(df)

In [14]:
path_test=f"gs://{'ornate-producer-329407'}/pyspark_nlp/CONSUMER_COMPLAINTS_TEST.csv"
path_test_class=f"gs://{'ornate-producer-329407'}/pyspark_nlp/labels_test.csv"

In [15]:
df1t=spark.read.csv(path_test, header=True)
df2t=spark.read.csv(path_test_class, header=True)

In [16]:
df_1t = df1t.withColumn("row_id", monotonically_increasing_id())
df_2t = df2t.withColumn("row_id", monotonically_increasing_id())
df_test = df_1t.join(df_2t, ("row_id")).drop("row_id")

In [17]:
res=pred.transform(df_test)

In [18]:
res.show()

22/11/24 18:53:59 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1295.4 KiB


+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               class|label|          token_text|         stop_tokens|               c_vec|              tf-idf|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|I purchase per pa...|        Prepaid card|  9.0|[i, purchase, per...|[purchase, per, p...|(8000,[18,19,25,2...|(8000,[18,19,25,2...|(8000,[18,19,25,2...|[-1173.9499553877...|[1.29368843478498...|       1.0|
|I paid my credit ...|         Credit card|  3.0|[i, paid, my, cre...|[paid, credit, ca...|(8000,[1,19,23,36...|(8000,[1,19,23,36...|(8000,[1,19,23,36...|[-1027.5557474

In [19]:
res=res.select('label','prediction')

In [20]:
res.write.csv("gs://ornate-producer-329407/output")

AnalysisException: path gs://ornate-producer-329407/output already exists.

In [ ]:
pred.write().overwrite().save("gs://ornate-producer-329407/NBmodel")